In [2]:
! pip install -q tensorflow-model-optimization

In [4]:
import numpy as np
import tensorflow as tf
import tensorflow 
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re

from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow_model_optimization.sparsity import keras as sparsity

# Custom Model on Mnist

In [5]:
#Load dataset as train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
num_classes = 10
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [7]:
from keras.models import Sequential
from keras import models, layers
from keras import regularizers
model = keras.Sequential()
model.add(keras.layers.Dropout(0.2,input_shape=(784,)))
model.add(keras.layers.Dense(1000,
                        kernel_regularizer = regularizers.l2(0.01),
                        activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1000,
                        kernel_regularizer = regularizers.l2(0.01),
                        activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10,  activation='softmax'))
#display the model summary
model.summary()

Using TensorFlow backend.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 1000)              785000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10010     
Total params: 1,796,010
Trainable params: 1,796,010
Non-trainable params: 0
______________________________________________

## (A) Post Training Quantization

In [8]:
model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer='adam', 
              metrics=['accuracy'])

In [9]:
hist = model.fit(x_train, y_train,
                        batch_size=128,
                        epochs=10,
                        verbose=1,
                        validation_data=(x_test,y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 77us/sample - loss: 2.0541 - accuracy: 0.8565 - val_loss: 0.6929 - val_accuracy: 0.9219
Epoch 2/10
60000/60000 [==============================] - 3s 52us/sample - loss: 0.7655 - accuracy: 0.8908 - val_loss: 0.6280 - val_accuracy: 0.9320
Epoch 3/10
60000/60000 [==============================] - 3s 52us/sample - loss: 0.7309 - accuracy: 0.8975 - val_loss: 0.6130 - val_accuracy: 0.9349
Epoch 4/10
60000/60000 [==============================] - 3s 52us/sample - loss: 0.7112 - accuracy: 0.9006 - val_loss: 0.6017 - val_accuracy: 0.9331
Epoch 5/10
60000/60000 [==============================] - 3s 53us/sample - loss: 0.6863 - accuracy: 0.9056 - val_loss: 0.5729 - val_accuracy: 0.9405
Epoch 6/10
60000/60000 [==============================] - 3s 52us/sample - loss: 0.6802 - accuracy: 0.9051 - val_loss: 0.5550 - val_accuracy: 0.9481
Epoch 7/10
60000/60000 [==============================] 

In [10]:
score = model.evaluate(x_test, y_test, verbose=1)
print("Test loss {:.4f}, accuracy {:.2f}%".format(score[0], score[1] * 100))

10000/10000 [==============================] - 1s 121us/sample - loss: 0.5148 - accuracy: 0.9492
Test loss 0.5148, accuracy 94.92%


In [11]:
#Save the entire model in model.h5 file
model.save("model.h5")
print("Saved model to disk")

Saved model to disk


In [12]:
model = tf.keras.models.load_model('model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
#saving converted model in "converted_model.tflite" file
open("converted_model.tflite", "wb").write(tflite_model)

7185624

In [13]:
model = tf.keras.models.load_model('model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
#saving converted model in "converted_quant_model.tflite" file
open("converted_quant_model.tflite", "wb").write(tflite_quant_model)

1803744

In [16]:
import os
print("Float model in Mb:", os.path.getsize('converted_model.tflite') / float(2**20))
print("Quantized model in Mb:", os.path.getsize('converted_quant_model.tflite') / float(2**20))
print("Compression ratio:", os.path.getsize('converted_model.tflite')/os.path.getsize('converted_quant_model.tflite'))

Float model in Mb: 6.852745056152344
Quantized model in Mb: 1.720184326171875
Compression ratio: 3.983727180797275


In [17]:
# Load TFLite model and allocate tensors.
interpreter = \
tf.lite.Interpreter(model_path="converted_quant_model.tflite")
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on some input data.
input_shape = input_details[0]['shape']
acc=0
for i in range(len(x_test)):
    input_data = x_test[i].reshape(input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    if(np.argmax(output_data) == np.argmax(y_test[i])):
        acc+=1
acc = acc/len(x_test)
print(acc*100)

95.6


## (B) Quantization Aware Training 

### Quantize full model

In [31]:
from keras.models import Sequential
from keras import models, layers
from keras import regularizers
model = keras.Sequential()
model.add(keras.layers.Dropout(0.2,input_shape=(784,)))
model.add(keras.layers.Dense(1000,
                        kernel_regularizer = regularizers.l2(0.01),
                        activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1000,
                        kernel_regularizer = regularizers.l2(0.01),
                        activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10,  activation='softmax'))
#display the model summary
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_12 (Dropout)         (None, 784)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1000)              785000    
_________________________________________________________________
dropout_13 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dropout_14 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                10010     
Total params: 1,796,010
Trainable params: 1,796,010
Non-trainable params: 0
____________________________________________

In [32]:
model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer='adam', 
              metrics=['accuracy'])

In [33]:
hist = model.fit(x_train, y_train,
                        batch_size=128,
                        epochs=100,
                        verbose=1,
                        validation_split = 0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/100
54000/54000 [==============================] - 4s 66us/sample - loss: 2.1969 - accuracy: 0.8506 - val_loss: 0.6505 - val_accuracy: 0.9327
Epoch 2/100
54000/54000 [==============================] - 3s 54us/sample - loss: 0.7705 - accuracy: 0.8896 - val_loss: 0.6030 - val_accuracy: 0.9443
Epoch 3/100
54000/54000 [==============================] - 3s 54us/sample - loss: 0.7433 - accuracy: 0.8951 - val_loss: 0.6016 - val_accuracy: 0.9423
Epoch 4/100
54000/54000 [==============================] - 3s 55us/sample - loss: 0.7206 - accuracy: 0.9001 - val_loss: 0.5755 - val_accuracy: 0.9528
Epoch 5/100
54000/54000 [==============================] - 3s 55us/sample - loss: 0.7063 - accuracy: 0.9022 - val_loss: 0.5372 - val_accuracy: 0.9603
Epoch 6/100
54000/54000 [==============================] - 3s 55us/sample - loss: 0.6881 - accuracy: 0.9041 - val_loss: 0.5492 - val_accuracy: 0.9567
Epoch 7/100
54000/54000 [==========================

In [35]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer='adam', 
              metrics=['accuracy'])

q_aware_model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: expected an indented block (<unknown>, line 14)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: expected an indented block (<unknown>, line 14)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: expected an indented block (<unknown>, line 14)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: expected an indented block (<unknown>, line 14)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and atta

In [40]:
histq = q_aware_model.fit(x_train, y_train,
                        batch_size=128,
                        epochs=100,
                        verbose=1,
                        validation_split = 0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/100
54000/54000 [==============================] - 4s 78us/sample - loss: 0.6439 - accuracy: 0.8917 - val_loss: 0.3743 - val_accuracy: 0.9677
Epoch 2/100
54000/54000 [==============================] - 4s 82us/sample - loss: 0.5447 - accuracy: 0.9074 - val_loss: 0.3573 - val_accuracy: 0.9682
Epoch 3/100
54000/54000 [==============================] - 4s 83us/sample - loss: 0.5390 - accuracy: 0.9082 - val_loss: 0.3609 - val_accuracy: 0.9657
Epoch 4/100
54000/54000 [==============================] - 4s 78us/sample - loss: 0.5355 - accuracy: 0.9097 - val_loss: 0.3540 - val_accuracy: 0.9677
Epoch 5/100
54000/54000 [==============================] - 4s 78us/sample - loss: 0.5415 - accuracy: 0.9083 - val_loss: 0.3755 - val_accuracy: 0.9632
Epoch 6/100
54000/54000 [==============================] - 4s 79us/sample - loss: 0.5401 - accuracy: 0.9084 - val_loss: 0.3655 - val_accuracy: 0.9648
Epoch 7/100
54000/54000 [==========================

In [41]:
_, baseline_model_accuracy = model.evaluate(
    x_test, y_test, verbose=1)

_, q_aware_model_accuracy = q_aware_model.evaluate(
    x_test, y_test, verbose=1)

print('Baseline test accuracy:', baseline_model_accuracy*100)
print('Quant test accuracy:', q_aware_model_accuracy*100)

10000/10000 [==============================] - 1s 140us/sample - loss: 0.3832 - accuracy: 0.9559
Baseline test accuracy: 95.6499993801117
Quant test accuracy: 95.59000134468079


In [36]:
#### fine tune with QAT on a subset of the training data.

In [37]:
histq= q_aware_model.fit(x_train[:1000], y_train[:1000],
                        batch_size=128,
                        epochs=100,
                        verbose=1,
                        validation_split = 0.1)

Train on 900 samples, validate on 100 samples
Epoch 1/100
900/900 [==============================] - 1s 1ms/sample - loss: 0.5572 - accuracy: 0.9100 - val_loss: 0.4607 - val_accuracy: 0.9500
Epoch 2/100
900/900 [==============================] - 0s 102us/sample - loss: 0.4868 - accuracy: 0.9189 - val_loss: 0.4986 - val_accuracy: 0.9400
Epoch 3/100
900/900 [==============================] - 0s 102us/sample - loss: 0.4185 - accuracy: 0.9267 - val_loss: 0.4676 - val_accuracy: 0.9400
Epoch 4/100
900/900 [==============================] - 0s 99us/sample - loss: 0.3880 - accuracy: 0.9378 - val_loss: 0.5742 - val_accuracy: 0.9400
Epoch 5/100
900/900 [==============================] - 0s 99us/sample - loss: 0.4207 - accuracy: 0.9289 - val_loss: 0.5365 - val_accuracy: 0.9500
Epoch 6/100
900/900 [==============================] - 0s 98us/sample - loss: 0.3750 - accuracy: 0.9444 - val_loss: 0.5380 - val_accuracy: 0.9400
Epoch 7/100
900/900 [==============================] - 0s 105us/sample - loss

In [39]:
_, baseline_model_accuracy = model.evaluate(
    x_test, y_test, verbose=1)

_, q_aware_model_accuracy = q_aware_model.evaluate(
    x_test, y_test, verbose=1)

print('Baseline test accuracy:', baseline_model_accuracy*100)
print('Quant test accuracy:', q_aware_model_accuracy*100)

10000/10000 [==============================] - 1s 143us/sample - loss: 0.6350 - accuracy: 0.9052
Baseline test accuracy: 95.6499993801117
Quant test accuracy: 90.52000045776367


### Quantize some layers

In [14]:
import tensorflow_model_optimization as tfmot
annotate = tfmot.quantization.keras.quantize_annotate_layer

In [15]:
from keras.models import Sequential
from keras import models, layers
from keras import regularizers
model = keras.Sequential()
model.add(keras.layers.Dropout(0.2,input_shape=(784,)))
model.add(keras.layers.Dense(1000,
                        kernel_regularizer = regularizers.l2(0.01),
                        activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(annotate(keras.layers.Dense(1000,
                        kernel_regularizer = regularizers.l2(0.01),
                        activation='relu')))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10,  activation='softmax'))
#display the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_3 (Dropout)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dropout_4 (Dropout)          (None, 1000)              0         
_________________________________________________________________
quantize_annotate_1 (Quantiz (None, 1000)              1001000   
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                10010     
Total params: 1,796,010
Trainable params: 1,796,010
Non-trainable params: 0
____________________________________________

In [16]:
# Use `quantize_apply` to actually make the model quantization aware.
quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: expected an indented block (<unknown>, line 14)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: expected an indented block (<unknown>, line 14)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_3 (Dropout)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dropout_4 (Dropout)          (None, 1000)              0         
_________________________________________________________________
quant_dense_4 (Quant

In [18]:
quant_aware_model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer='adam', 
              metrics=['accuracy'])


In [22]:
histq = quant_aware_model.fit(x_train, y_train,
                        batch_size=128,
                        epochs=100,
                        verbose=1,
                        validation_split = 0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/100
54000/54000 [==============================] - 4s 68us/sample - loss: 2.1874 - accuracy: 0.8525 - val_loss: 0.6378 - val_accuracy: 0.9402
Epoch 2/100
54000/54000 [==============================] - 2s 38us/sample - loss: 0.7794 - accuracy: 0.8882 - val_loss: 0.6252 - val_accuracy: 0.9367
Epoch 3/100
54000/54000 [==============================] - 2s 37us/sample - loss: 0.7430 - accuracy: 0.8960 - val_loss: 0.6064 - val_accuracy: 0.9448
Epoch 4/100
54000/54000 [==============================] - 2s 39us/sample - loss: 0.7241 - accuracy: 0.8996 - val_loss: 0.5572 - val_accuracy: 0.9543
Epoch 5/100
54000/54000 [==============================] - 2s 38us/sample - loss: 0.7001 - accuracy: 0.9026 - val_loss: 0.5618 - val_accuracy: 0.9490
Epoch 6/100
54000/54000 [==============================] - 2s 37us/sample - loss: 0.6919 - accuracy: 0.9021 - val_loss: 0.5340 - val_accuracy: 0.9577
Epoch 7/100
54000/54000 [==========================

In [24]:
_, quant_aware_model_accuracy = quant_aware_model.evaluate(
    x_test, y_test, verbose=1)

print('Quant test accuracy:', quant_aware_model_accuracy*100)

10000/10000 [==============================] - 1s 85us/sample - loss: 0.4013 - accuracy: 0.9563
Quant test accuracy: 95.63000202178955


# VGG on CIFAR10

In [5]:
from keras.datasets import cifar10
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

Using TensorFlow backend.


170500096/170498071 [==============================] - 4s 0us/step


In [6]:
vgg_model = tf.keras.applications.VGG16(input_shape=x_train[0].shape, include_top=False, weights=None)
vgg_model.trainable = True
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

model = keras.Sequential()
model.add(vgg_model)
model.add(global_average_layer)
model.add(keras.layers.Dense(1024, activation='relu'))
#model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10250     
Total params: 15,250,250
Trainable params: 535,562
Non-trainable params: 14,714,688
_________________________________________________________________
None


In [7]:
from tensorflow.keras import losses
#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='adam', 
              loss=losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

In [8]:
hist = model.fit(x_train, y_train,
                        batch_size=128,
                        epochs=20,
                        verbose=1,
                        validation_split=0.1)

Train on 45000 samples, validate on 5000 samples
Epoch 1/20
45000/45000 [==============================] - 20s 449us/sample - loss: 2.0840 - accuracy: 0.2348 - val_loss: 1.9510 - val_accuracy: 0.2824
Epoch 2/20
45000/45000 [==============================] - 14s 314us/sample - loss: 1.8850 - accuracy: 0.3179 - val_loss: 1.8595 - val_accuracy: 0.3204
Epoch 3/20
45000/45000 [==============================] - 14s 315us/sample - loss: 1.8158 - accuracy: 0.3437 - val_loss: 1.7738 - val_accuracy: 0.3648
Epoch 4/20
45000/45000 [==============================] - 14s 317us/sample - loss: 1.7637 - accuracy: 0.3640 - val_loss: 1.7309 - val_accuracy: 0.3668
Epoch 5/20
45000/45000 [==============================] - 14s 318us/sample - loss: 1.7258 - accuracy: 0.3780 - val_loss: 1.7263 - val_accuracy: 0.3758
Epoch 6/20
45000/45000 [==============================] - 14s 321us/sample - loss: 1.7037 - accuracy: 0.3912 - val_loss: 1.6905 - val_accuracy: 0.3842
Epoch 7/20
45000/45000 [=====================

In [9]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss {:.4f}, accuracy {:.2f}%".format(score[0], score[1] * 100))

Test loss 1.5641, accuracy 44.35%


In [10]:
model.save("vgg_model.h5")
print("Saved model to disk")

Saved model to disk


In [11]:
model = tf.keras.models.load_model('vgg_model.h5')
model.compile(optimizer='adam', 
              loss=losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])
hist = model.fit(x_train, y_train,
                        batch_size=128,
                        epochs=50,
                        verbose=1,
                        validation_split=0.1)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 15s 337us/sample - loss: 1.5492 - accuracy: 0.4503 - val_loss: 1.5598 - val_accuracy: 0.4420
Epoch 2/50
45000/45000 [==============================] - 14s 317us/sample - loss: 1.5499 - accuracy: 0.4491 - val_loss: 1.5494 - val_accuracy: 0.4464
Epoch 3/50
45000/45000 [==============================] - 14s 318us/sample - loss: 1.5463 - accuracy: 0.4498 - val_loss: 1.5951 - val_accuracy: 0.4350
Epoch 4/50
45000/45000 [==============================] - 14s 319us/sample - loss: 1.5398 - accuracy: 0.4518 - val_loss: 1.5661 - val_accuracy: 0.4380
Epoch 5/50
45000/45000 [==============================] - 14s 319us/sample - loss: 1.5320 - accuracy: 0.4558 - val_loss: 1.5503 - val_accuracy: 0.4480
Epoch 6/50
45000/45000 [==============================] - 14s 319us/sample - loss: 1.5297 - accuracy: 0.4565 - val_loss: 1.5489 - val_accuracy: 0.4376
Epoch 7/50
45000/45000 [=====================

In [12]:
model = tf.keras.models.load_model('vgg_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
open("vgg_quant_model.tflite", "wb").write(tflite_quant_model)

15259960

In [13]:
x_test = x_test.astype('float32')

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="vgg_quant_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on some input data.
input_shape = input_details[0]['shape']
acc=0
for i in range(len(x_test)):
    input_data = x_test[i].reshape(input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    if(np.argmax(output_data) == y_test[i]):
        acc+=1
acc = acc/len(x_test)
print(acc*100)

75.64


In [15]:
import os
import tempfile

# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(tflite_quant_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

Float model in Mb: 58.182395935058594
Quantized model in Mb: 14.566337585449219


## QAT

### Quantize full model

In [51]:
import tensorflow as tf

In [54]:
vgg_model = tf.keras.applications.VGG16(input_shape=(32, 32, 3), include_top=False, weights=None)
vgg_model.trainable = True
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

#https://github.com/tensorflow/model-optimization/issues/40    
model = tf.keras.Sequential()
#Had to add input layer, otherwise ran into trouble when printing model summary as well as converting to tf-lite
model.add(layers.Input(shape=(32, 32, 3)))
input_layer = True
for layer in vgg_model.layers:
    model.add(layer)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
#model.add(layers.BatchNormalization())
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        multiple                  0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)        

In [55]:
model.compile(loss=losses.sparse_categorical_crossentropy, 
              optimizer='adam', 
              metrics=['accuracy'])

hist= model.fit(x_train, y_train,
                        batch_size=128,
                        epochs=20,
                        verbose=1,
                        validation_split = 0.1)

Train on 45000 samples, validate on 5000 samples
Epoch 1/20
45000/45000 [==============================] - 42s 944us/sample - loss: 2.3878 - accuracy: 0.1385 - val_loss: 1.9351 - val_accuracy: 0.2294
Epoch 2/20
45000/45000 [==============================] - 41s 922us/sample - loss: 1.7715 - accuracy: 0.2946 - val_loss: 1.6209 - val_accuracy: 0.3746
Epoch 3/20
45000/45000 [==============================] - 42s 922us/sample - loss: 1.5206 - accuracy: 0.4189 - val_loss: 1.3773 - val_accuracy: 0.4782
Epoch 4/20
45000/45000 [==============================] - 42s 923us/sample - loss: 1.2979 - accuracy: 0.5241 - val_loss: 1.1892 - val_accuracy: 0.5740
Epoch 5/20
45000/45000 [==============================] - 42s 923us/sample - loss: 1.1204 - accuracy: 0.6008 - val_loss: 1.0787 - val_accuracy: 0.6208
Epoch 6/20
45000/45000 [==============================] - 41s 922us/sample - loss: 0.9860 - accuracy: 0.6480 - val_loss: 0.9629 - val_accuracy: 0.6546
Epoch 7/20
45000/45000 [=====================

In [56]:
model.save("vgg_true_model.h5")
print("Saved model to disk")

Saved model to disk


In [57]:
import tensorflow_model_optimization as tfmot
#model = tf.keras.models.load_model('vgg_model.h5')
quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(loss=losses.sparse_categorical_crossentropy, 
              optimizer='adam', 
              metrics=['accuracy'])

model.compile(loss=losses.sparse_categorical_crossentropy, 
              optimizer='adam', 
              metrics=['accuracy'])

q_aware_model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: expected an indented block (<unknown>, line 14)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: expected an indented block (<unknown>, line 14)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: expected an indented block (<unknown>, line 14)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: expected an indented block (<unknown>, line 14)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and atta

In [58]:
histq= q_aware_model.fit(x_train[:1000], y_train[:1000],
                        batch_size=128,
                        epochs=10,
                        verbose=1,
                        validation_split = 0.1)

Train on 900 samples, validate on 100 samples
Epoch 1/10
900/900 [==============================] - 7s 8ms/sample - loss: 1.9251 - accuracy: 0.3978 - val_loss: 1.2186 - val_accuracy: 0.6200
Epoch 2/10
900/900 [==============================] - 4s 5ms/sample - loss: 1.1039 - accuracy: 0.6478 - val_loss: 1.0042 - val_accuracy: 0.6800
Epoch 3/10
900/900 [==============================] - 4s 5ms/sample - loss: 0.6566 - accuracy: 0.7844 - val_loss: 0.6959 - val_accuracy: 0.8100
Epoch 4/10
900/900 [==============================] - 4s 5ms/sample - loss: 0.3488 - accuracy: 0.9100 - val_loss: 0.9124 - val_accuracy: 0.7500
Epoch 5/10
900/900 [==============================] - 4s 5ms/sample - loss: 0.2066 - accuracy: 0.9444 - val_loss: 0.8089 - val_accuracy: 0.7800
Epoch 6/10
900/900 [==============================] - 4s 5ms/sample - loss: 0.1489 - accuracy: 0.9611 - val_loss: 0.9827 - val_accuracy: 0.7800
Epoch 7/10
900/900 [==============================] - 4s 5ms/sample - loss: 0.1971 - accur

In [59]:
_, baseline_model_accuracy = model.evaluate(
    x_test, y_test, verbose=1)

_, q_aware_model_accuracy = q_aware_model.evaluate(
    x_test, y_test, verbose=1)

print('Baseline test accuracy:', baseline_model_accuracy*100)
print('Quant test accuracy:', q_aware_model_accuracy*100)

10000/10000 [==============================] - 35s 3ms/sample - loss: 1.6151 - accuracy: 0.6405
Baseline test accuracy: 74.27999973297119
Quant test accuracy: 64.0500009059906


In [60]:
q_aware_model.save("vgg_qat_full_model.h5")
print("Saved model to disk")


Saved model to disk


In [61]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
open("vgg_qat_full_model.tflite", "wb").write(tflite_quant_model)

15277856

In [65]:
print("Float complete model in Mb:", os.stat('vgg_true_model.h5').st_size/ float(2**20))
print("QAT model in Mb:", os.stat('vgg_qat_full_model.h5').st_size/ float(2**20))
print("QAT compressed model in Mb:", os.stat('vgg_qat_full_model.tflite').st_size/ float(2**20))

Float complete model in Mb: 174.66024017333984
QAT model in Mb: 174.7544708251953
QAT compressed model in Mb: 14.570098876953125


In [67]:
import os
import tempfile

# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(tflite_quant_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float tflite model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

Float tflite model in Mb: 58.18195343017578
Quantized model in Mb: 14.570098876953125


In [68]:
x_test = x_test.astype('float32')

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="vgg_qat_full_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on some input data.
input_shape = input_details[0]['shape']
acc=0
for i in range(len(x_test)):
    input_data = x_test[i].reshape(input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    if(np.argmax(output_data) == y_test[i]):
        acc+=1
acc = acc/len(x_test)
print(acc*100)

64.25


### Quantize some layer

In [19]:
annotate = tfmot.quantization.keras.quantize_annotate_layer

In [20]:
vgg_model = tf.keras.applications.VGG16(input_shape=(32, 32, 3), include_top=False, weights=None)
vgg_model.trainable = True
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

#https://github.com/tensorflow/model-optimization/issues/40    
model = tf.keras.Sequential()
#Had to add input layer, otherwise ran into trouble when printing model summary as well as converting to tf-lite
model.add(layers.Input(shape=(32, 32, 3)))
input_layer = True
for layer in vgg_model.layers:
    model.add(layer)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
#model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(annotate(layers.Dense(10, activation='softmax')))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         multiple                  0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)        

In [21]:
# Use `quantize_apply` to actually make the model quantization aware.
quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: expected an indented block (<unknown>, line 14)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: expected an indented block (<unknown>, line 14)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D

In [22]:
histq= q_aware_model.fit(x_train[:1000], y_train[:1000],
                        batch_size=128,
                        epochs=10,
                        verbose=1,
                        validation_split = 0.1)

Train on 900 samples, validate on 100 samples
Epoch 1/10
900/900 [==============================] - 5s 6ms/sample - loss: 0.0744 - accuracy: 0.9767 - val_loss: 1.0120 - val_accuracy: 0.7800
Epoch 2/10
900/900 [==============================] - 4s 5ms/sample - loss: 0.1263 - accuracy: 0.9678 - val_loss: 1.2246 - val_accuracy: 0.7200
Epoch 3/10
900/900 [==============================] - 4s 5ms/sample - loss: 0.0924 - accuracy: 0.9756 - val_loss: 1.1501 - val_accuracy: 0.7500
Epoch 4/10
900/900 [==============================] - 4s 5ms/sample - loss: 0.0842 - accuracy: 0.9778 - val_loss: 1.5330 - val_accuracy: 0.7400
Epoch 5/10
900/900 [==============================] - 4s 5ms/sample - loss: 0.0507 - accuracy: 0.9844 - val_loss: 1.3601 - val_accuracy: 0.7500
Epoch 6/10
900/900 [==============================] - 4s 5ms/sample - loss: 0.0805 - accuracy: 0.9800 - val_loss: 1.3084 - val_accuracy: 0.7000
Epoch 7/10
900/900 [==============================] - 4s 5ms/sample - loss: 0.0652 - accur

In [18]:
_, baseline_model_accuracy = model.evaluate(
    x_test, y_test, verbose=1)

_, q_aware_model_accuracy = q_aware_model.evaluate(
    x_test, y_test, verbose=1)

print('Baseline test accuracy:', baseline_model_accuracy*100)
print('Quant test accuracy:', q_aware_model_accuracy*100)

10000/10000 [==============================] - 47s 5ms/sample - loss: 1.5860 - accuracy: 0.6652
Baseline test accuracy: 73.94000291824341
Quant test accuracy: 66.51999950408936


In [27]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
open("vgg_qat_model.tflite", "wb").write(tflite_quant_model)

15277840

In [28]:
import os
import tempfile

# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(tflite_quant_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

Float model in Mb: 58.168704986572266
Quantized model in Mb: 14.570083618164062


In [39]:
print("Float complete model in Mb:", os.stat('vgg_model.h5').st_size/ float(2**20))
print("QAT compressed model in Mb:", os.stat('vgg_qat_model.tflite').st_size/ float(2**20))

Float complete model in Mb: 62.32586669921875
QAT compressed model in Mb: 14.570083618164062


In [29]:
x_test = x_test.astype('float32')

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="vgg_qat_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on some input data.
input_shape = input_details[0]['shape']
acc=0
for i in range(len(x_test)):
    input_data = x_test[i].reshape(input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    if(np.argmax(output_data) == y_test[i]):
        acc+=1
acc = acc/len(x_test)
print(acc*100)

65.42
